In [1]:
import pandas as pd
import numpy as np

meta_json_data_path = '../../data/preprocessed/meta.json'

meta = pd.read_json(meta_json_data_path)
meta.head()

,filename,meta,ids,pos_tag,folder,document
0,d05-040,"[Or, when, he, found, cock-fighting, going, on...","[d05-040-00-00, d05-040-00-01, d05-040-00-02, ...","[CC, WRB, PP3A, VBD, NN, VBG, RP, IN, NN, ?, N...",d05,d05-040
1,p03-151,"["", Isn't, it, ?, "", he, rose, and, smoothed, ...","[p03-151-00-00, p03-151-00-01, p03-151-00-02, ...","[*', BEZX, PP3, ?, **', PP3A, VBD, CC, VBD, PP...",p03,p03-151
10,c02-012,"[Every, unnecessary, detail, ,, all, superfluo...","[c02-012-00-00, c02-012-00-01, c02-012-00-02, ...","[AT, JJ, NN, ,, ABN, JJ, JJ, NN, MD, BE, VBN, ...",c02,c02-012
100,g06-047n,"[This, remarkable, man, of, medicine, ,, whom,...","[g06-047n-00-00, g06-047n-00-01, g06-047n-00-0...","[DT, JJ, NN, INO, NN, ,, WPO, NPT, NP, NP, VBD...",g06,g06-047
1000,a05-029,"[Sir, Francis, is, to, hand, over, to, Sir, Ri...","[a05-029-00-00, a05-029-00-01, a05-029-00-02, ...","[NPT, NP, BEZ, TO, VB, RP, IN, NPT, NP, IN21, ...",a05,a05-029


In [83]:
threshold_length = 3

from nltk.tokenize import sent_tokenize
import random

X = []
y = []
for sent in sent_tokenize(' '.join(meta.meta[0])):
    split_sent = sent.split()
    found = True
    while found == True:
        random_int = random.randint(0, len(split_sent)-1)
        target = split_sent[random_int]
        if len(target) >= threshold_length:
            print(split_sent)
            print(split_sent[:random_int])
            print(target)
            print(split_sent[random_int+1:])
            X.append(split_sent[:random_int] + split_sent[random_int+1:])
            y.append((target, random_int))
            found = False
            

['Or', 'when', 'he', 'found', 'cock-fighting', 'going', 'on', 'in', 'church', '?']
['Or', 'when', 'he', 'found', 'cock-fighting', 'going', 'on', 'in']
church
['?']
['Laud', 'made', 'a', 'positive', 'approach', '.']
['Laud', 'made', 'a']
positive
['approach', '.']
['He', 'set', 'out', 'to', 'increase', 'a', 'sense', 'of', 'reverence', '.']
['He', 'set', 'out', 'to']
increase
['a', 'sense', 'of', 'reverence', '.']
['The', 'Prayer', 'Book', 'was', 'to', 'be', 'respected', 'and', 'so', 'was', 'the', 'office', 'of', 'a', 'bishop', '.']
['The']
Prayer
['Book', 'was', 'to', 'be', 'respected', 'and', 'so', 'was', 'the', 'office', 'of', 'a', 'bishop', '.']
['Altars', 'should', 'be', 'altars', 'and', 'not', 'any', 'broken', 'down', ',', 'transportable', 'table', 'which', 'was', 'handy', 'for', 'the', 'most', 'improbable', 'uses', '.']
['Altars', 'should', 'be', 'altars', 'and', 'not', 'any', 'broken', 'down', ',', 'transportable', 'table', 'which', 'was', 'handy', 'for']
the
['most', 'improbable

In [84]:
y

[('church', 8), ('positive', 3), ('increase', 4), ('Prayer', 1), ('the', 16)]

In [85]:
X

[['Or', 'when', 'he', 'found', 'cock-fighting', 'going', 'on', 'in', '?'],
 ['Laud', 'made', 'a', 'approach', '.'],
 ['He', 'set', 'out', 'to', 'a', 'sense', 'of', 'reverence', '.'],
 ['The',
  'Book',
  'was',
  'to',
  'be',
  'respected',
  'and',
  'so',
  'was',
  'the',
  'office',
  'of',
  'a',
  'bishop',
  '.'],
 ['Altars',
  'should',
  'be',
  'altars',
  'and',
  'not',
  'any',
  'broken',
  'down',
  ',',
  'transportable',
  'table',
  'which',
  'was',
  'handy',
  'for',
  'most',
  'improbable',
  'uses',
  '.']]

In [ ]:
def ptb_iterator(raw_data, batch_size, num_steps, epochs, steps_ahead=1):

	data = np.array(raw_data)
	data_len = data.shape[0]
	nb_batches = (data_len - 1) // (batch_size * num_steps)
	assert nb_batches > 0, "Not enough data, even for a single batch. Try using a smaller batch_size."
	rounded_data_len = nb_batches * batch_size * num_steps
	xdata = np.reshape(data[0:rounded_data_len], [batch_size, nb_batches * num_steps])
	ydata = np.reshape(data[1:rounded_data_len + 1], [batch_size, nb_batches * num_steps])

	for epoch in range(epochs):
		for batch in range(nb_batches):
			x = xdata[:, batch * num_steps:(batch + 1) * num_steps]
			y = ydata[:, batch * num_steps:(batch + 1) * num_steps]
			x = np.roll(x, -epoch, axis=0)  
			y = np.roll(y, -epoch, axis=0)
			yield x, y, epoc